# scikit-mobility
### scikit-mobilityは位置情報を可視化するツールで、人の動きについて解析することができる。

In [8]:
import skmob
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [3]:
tdf = skmob.TrajDataFrame.from_file('geolife_sample.txt.gz', latitude='lat', longitude='lon', user_id='user', datetime='datetime')
tdf.head()

,lat,lng,datetime,uid
0,39.984094,116.319236,2008-10-23 05:53:05,1
1,39.984198,116.319322,2008-10-23 05:53:06,1
2,39.984224,116.319402,2008-10-23 05:53:11,1
3,39.984211,116.319389,2008-10-23 05:53:16,1
4,39.984217,116.319422,2008-10-23 05:53:21,1


In [11]:
geo_plot = tdf.plot_trajectory(zoom=10, weight=3, opacity=0.9, tiles='OpenStreetMap')
geo_plot.savefig('[4]実行.png')

AttributeError: 'Map' object has no attribute 'savefig'

# フィルター機能で適切な可視化
### filteringを用いる
### max_speed_kmが５(km/h)から上の図と比較すると下の線が短くなっていることが分かる

In [5]:
from skmob.preprocessing import filtering
filtered_tdf = filtering.filter(tdf, max_speed_kmh=5)
filtered_tdf.plot_trajectory(zoom=10, weight=3, opacity=0.9, tiles='OpenStreetMap')

# 滞在検出機能を用いてプロット
### detectionを用いる
### stop_radius_factorとspatial_radius_kmに指定した時間(minutes_for_a_stop分間)いた人をプロットすることができる

In [6]:
import skmob
from skmob.preprocessing import detection
import pandas as pd

url = skmob.utils.constants.GEOLIFE_SAMPLE
df = pd.read_csv(url, sep=',', compression='gzip')
tdf = skmob.TrajDataFrame(df, latitude='lat', longitude='lon', user_id='user', datetime='datetime')

stdf = detection.stay_locations(tdf, stop_radius_factor=0.5, minutes_for_a_stop=60, spatial_radius_km=0.2, leaving_time=True)

map_f = tdf.plot_trajectory(max_points=1000, start_end_markers=False)
stdf.plot_stops(map_f=map_f)

## FlowDataFrameを使うと下記の地図を出力することも可能。

In [7]:
import skmob
import geopandas as gpd
# load a spatial tessellation
url_tess = skmob.utils.constants.NY_COUNTIES_2011
tessellation = gpd.read_file(url_tess).rename(columns={'tile_id': 'tile_ID'})
# load real flows into a FlowDataFrame
# download the file with the real fluxes from: https://raw.githubusercontent.com/scikit-mobility/scikit-mobility/master/tutorial/data/NY_commuting_flows_2011.csv
fdf = skmob.FlowDataFrame.from_file("NY_commuting_flows_2011.csv",
                                        tessellation=tessellation,
                                        tile_id='tile_ID',
                                        sep=",")
m = fdf.plot_flows(flow_color='purple', tiles='OpenStreetMap')
m